In [34]:
import numpy as np

In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType, IntegerType, DoubleType
from pyspark.sql.functions import *
import utils
import numeric_outliers

In [2]:
spark = SparkSession \
        .builder \
        .appName("Nulls and Outliers Detection 1") \
        .getOrCreate()


In [3]:
df_temp = df = spark.read.csv(path = 'GROUP7/bss9-579f_clean/part-00000-919430be-1ef9-4fee-9d5f-c8cac19198b3-c000.csv', header = True,inferSchema = True, sep='\t')

In [6]:
df_count = df_temp.count()

In [18]:
keyval = {}
df_temp = df_temp.withColumn("rid", monotonically_increasing_id())
for col,dtype in df_temp.dtypes:
    if 'string' not in dtype and col!='rid':
        
        keyval[col] = numeric_outliers.get_outliers(df_temp,col)
        print(col , "  counts:",keyval[col].count(),'  counts % = {}%'.format(100*(keyval[col].count()/df_count)))

comparable_rental_2_gross_sqft   counts: 45   counts % = 9.375%
brooklyn_condominiums_comparable_properties_estimated_expense   counts: 46   counts % = 9.583333333333334%
brooklyn_condominiums_comparable_properties_total_units   counts: 48   counts % = 10.0%
brooklyn_condominiums_comparable_properties_expense_per_sqft   counts: 5   counts % = 1.0416666666666665%
comparable_rental_2_gross_income_per_sqft   counts: 9   counts % = 1.875%
comparable_rental_1_gross_income_per_sqft   counts: 4   counts % = 0.8333333333333334%
comparable_rental_1_estimated_expense   counts: 53   counts % = 11.041666666666666%
comparable_rental_1_net_operating_income   counts: 63   counts % = 13.125%
comparable_rental_2_estimated_expense   counts: 52   counts % = 10.833333333333334%
brooklyn_condominiums_comparable_properties_gross_sqft   counts: 45   counts % = 9.375%
brooklyn_condominiums_comparable_properties_market_value_per_sqft   counts: 3   counts % = 0.625%
brooklyn_condominiums_comparable_properties_g

In [19]:
from pyspark.mllib.clustering import KMeans, KMeansModel

In [62]:
df_col = df_temp.select(df_temp['rid'],df_temp['comparable_rental_2_gross_sqft'])

In [23]:
df_vect = df_temp.map(lambda x: np.array(float(x['comparable_rental_2_gross_sqft'])))

AttributeError: 'DataFrame' object has no attribute 'map'

In [69]:
df_temp.select(df_temp['rid'] == 0,df_temp['comparable_rental_2_gross_sqft']).show()

+---------+------------------------------+
|(rid = 0)|comparable_rental_2_gross_sqft|
+---------+------------------------------+
|     true|                        109875|
|    false|                          9875|
|    false|                         39770|
|    false|                         39770|
|    false|                         39770|
|    false|                        108780|
|    false|                         28358|
|    false|                          9875|
|    false|                         28358|
|    false|                         31692|
|    false|                         60720|
|    false|                         73032|
|    false|                         39770|
|    false|                         39770|
|    false|                         39770|
|    false|                         15200|
|    false|                         27144|
|    false|                          8600|
|    false|                         21040|
|    false|                          8600|
+---------+

In [65]:
df_col_rdd.take(5)

[Row(rid=0, comparable_rental_2_gross_sqft=109875),
 Row(rid=1, comparable_rental_2_gross_sqft=9875),
 Row(rid=2, comparable_rental_2_gross_sqft=39770),
 Row(rid=3, comparable_rental_2_gross_sqft=39770),
 Row(rid=4, comparable_rental_2_gross_sqft=39770)]

In [70]:
vso = df_col_rdd.map(lambda x: np.array(float(x[1])))

In [75]:
clusters = KMeans.train(vso,3,initializationMode='random',maxIterations=100)

In [79]:
len(clusters.centers)

3

In [80]:
def addclustercols(x):
    point = np.array(float(x[1]))
    center = clusters.centers[0]
    mindist = np.abs(point - center)
    c1 = 0
    for i in range(1,len(clusters.centers)):
        center = clusters.centers[i]
        dist = np.abs(point - center)
        if dist < mindist:
            c1 = i
            mindist = dist
    return (int(x[0]),float(x[1]),int(c1),float(mindist))

In [81]:
rdd_w_clusts = df_col_rdd.map(lambda x: addclustercols(x))

In [83]:
col = 'comparable_rental_2_gross_sqft'

In [84]:
kmeans_df = spark.createDataFrame(rdd_w_clusts,('rid',col,'c_no','dist_c'))

In [85]:
outliers = numeric_outliers.get_outliers(kmeans_df,'dist_c')

In [87]:
outliers.count()/df

16

In [26]:
sc = spark.sparkContext


In [27]:
file_data = sc.textFile('GROUP7/bss9-579f_clean/part-00000-919430be-1ef9-4fee-9d5f-c8cac19198b3-c000.csv')